<center><img src="src/ipywidgets.svg" width="50%"></center>

# Repository: https://github.com/jupyter-widgets/ipywidgets
# Installation: 
`conda install -c conda-forge ipywidgets`

## Simple slider for driving an integer value

In [2]:
from ipywidgets import IntSlider

In [3]:
slider = IntSlider()

In [4]:
slider

IntSlider(value=0)

In [5]:
slider.value

34

In [6]:
slider.value = 20

In [7]:
slider

IntSlider(value=20)

## Widgets protocol

<center><img src="src/widgets-arch.png" width="50%"></center>

## Drive a boolean value

In [8]:
from ipywidgets import Checkbox

In [9]:
checkbox = Checkbox(description='Check me')
checkbox

Checkbox(value=False, description='Check me')

In [10]:
checkbox.value

False

In [11]:
checkbox.value = False

## Link two widgets

In [12]:
from ipywidgets import IntText, IntSlider, link, HBox

In [13]:
text = IntText()

slider = IntSlider()

link((text, 'value'), (slider, 'value'))
HBox([text, slider])

## Observe changes on the widget model

In [14]:
from ipywidgets import ToggleButton

In [16]:
button = ToggleButton(description='Click me!', button_style='danger')

def update_style(change):
    button.button_style = 'info' if change['new'] else 'danger'

button.observe(update_style, 'value')
button

ToggleButton(value=False, button_style='danger', description='Click me!')

## Variety of widgets in the core library

In [17]:
from ipywidgets import ColorPicker, DatePicker, IntProgress, Play, VBox, link

In [18]:
progress = IntProgress()
play = Play()
link((play, 'value'), (progress, 'value'))

VBox([ColorPicker(value='red'), DatePicker(), progress, play])

## Media widgets

### Image widget

In [19]:
from ipywidgets import Image

import PIL.Image
import io
import numpy as np
from skimage.filters import sobel
from skimage.color.adapt_rgb import adapt_rgb, each_channel
from skimage import filters

/home/jan-hendrik/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [20]:
image = Image.from_file("src/marie.png")
image

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x18\x00\x00\x01\x8c\x08\x02\x00\x00\x00\x8e\x1b*…

In [21]:
im_in = PIL.Image.open(io.BytesIO(image.value))
im_array = np.array(im_in)[...,:3]
im_array

array([[[73, 73, 73],
        [71, 71, 71],
        [66, 66, 66],
        ...,
        [80, 80, 80],
        [81, 81, 81],
        [80, 80, 80]],

       [[71, 71, 71],
        [78, 78, 78],
        [76, 76, 76],
        ...,
        [73, 73, 73],
        [78, 78, 78],
        [83, 83, 83]],

       [[76, 76, 76],
        [78, 78, 78],
        [83, 83, 83],
        ...,
        [75, 75, 75],
        [83, 83, 83],
        [91, 91, 91]],

       ...,

       [[ 0,  0,  0],
        [ 1,  1,  1],
        [ 5,  5,  5],
        ...,
        [71, 71, 71],
        [73, 73, 73],
        [73, 73, 73]],

       [[ 0,  0,  0],
        [ 1,  1,  1],
        [ 4,  4,  4],
        ...,
        [76, 76, 76],
        [75, 75, 75],
        [74, 74, 74]],

       [[ 1,  1,  1],
        [ 1,  1,  1],
        [ 1,  1,  1],
        ...,
        [77, 77, 77],
        [77, 77, 77],
        [72, 72, 72]]], dtype=uint8)

In [22]:
im_array_edges = adapt_rgb(each_channel)(sobel)(im_array)
im_array_edges = ((1-im_array_edges) * 255).astype(np.uint8)
im_out = PIL.Image.fromarray(im_array_edges)
f = io.BytesIO()
im_out.save(f, format='png')
image.value = f.getvalue()
image

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x18\x00\x00\x01\x8c\x08\x02\x00\x00\x00\x8e\x1b*…

### Video widget

In [23]:
from ipywidgets import Video, Image
from IPython.display import display
import numpy as np
import cv2
import base64

In [24]:
video = Video.from_file('src/Big.Buck.Bunny.mp4')
video

Video(value=b'\x00\x00\x00\x18ftypisom\x00\x00\x00\x00mp41avc1\x00\x00\x1bNmoov\x00\x00\x00lmvhd\x00\x00\x00\x…

In [25]:
cap = cv2.VideoCapture('src/Big.Buck.Bunny.mp4')

frames = []

while(1):
    try:
        _, frame = cap.read()

        fgmask = cv2.Canny(frame, 100, 100)

        mask = fgmask > 100
        frame[mask, :] = 0

        frames.append(frame)
    except Exception:
        break

width = int(cap.get(3))
height = int(cap.get(4))

filename = 'src/output.mp4'

fourcc = cv2.VideoWriter_fourcc(*'avc1')
writer = cv2.VideoWriter(filename, fourcc, 25, (width, height))

for frame in frames:
    writer.write(frame)

cap.release()
writer.release()

with open(filename, 'rb') as f:
    video.value = f.read()

### Clean

In [26]:
from ipywidgets import Widget
Widget.close_all()